<a href="https://colab.research.google.com/github/hinsley/RL-depot/blob/master/temporal-difference/TDc4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TDc4

Temporal difference learning model for Connect 4

---

Uses value function approximation inspired by [this paper](https://link.springer.com/content/pdf/10.1007/s10994-012-5280-0.pdf)

State features: 1x1, 2x2, 3x3, 4x4, 5x5 local configurations, reflected horizontally for weight sharing, with location dependent and location independent weight vectors

---

### To Do:

- [X] Add parallelization
  - [X] Afterstate evaluation
  - [X] Rollouts
  - [X] Training
- [X] Make rollouts update immediate afterstate values
- [ ] Save a strong pre-trained weight vector
- [ ] Implement equivalence classes / weight sharing

In [0]:
import math
import multiprocessing as mp
import numpy as np
import requests
from collections import Counter
from functools import partial
from itertools import count
from random import choice, choices
from time import time
from typing import Dict, List, Optional, Tuple, Union

In [0]:
PositionState = int # See below.
VALUE_BLANK = 0
VALUE_X = 1
VALUE_O = 2

GameState = np.ndarray # 6 x 7 (Rows x Cols)

LocalFeature = np.ndarray # [1, 5] x [1, 5] (Rows x Cols)

EncodedFeature = int # Does not include any indication of local feature size.

# Indexed by feature size, and row/column of upper-leftmost position. Row and column are -1 for location-independent features.
EncodedFeatureVector = Dict[int, Dict[int, Dict[int, List[EncodedFeature]]]]

# Indexed by feature size, and row/column of upper-leftmost position. Row and column are -1 for location-independent features.
WeightVector = Dict[int, Dict[int, Dict[int, Dict[EncodedFeature, float]]]]

StateValue = float # [0.0, 1.0]

In [0]:
def pprint(state: GameState):
  """ Displays a given game configuration. """
  def rasterize_position(position_state: PositionState) -> str:
    return {
        VALUE_BLANK: " ",
        VALUE_X: "X",
        VALUE_O: "O",
    }[position_state]

  print(" __ _ _ _ _ _ __")
  for i, row in enumerate(state):
    print(f"""{i+1}|{' '.join([rasterize_position(position_state) for
                              position_state in
                              row])}|""")
  print(" -- - - - - - --")
  print("  A B C D E F G ")

In [0]:
def empty_game_state() -> GameState:
  """ Generates an empty, new game state. """
  rows = 6
  cols = 7
  return np.full((rows, cols), VALUE_BLANK)

In [0]:
def eval_state(state: GameState, check_game_over: bool=False) -> Union[StateValue, bool]:
  """
  Takes as input a game state and returns a reward (for X winning), punishment
  (for X losing), or a neutral state value (unexplored ambiguity). This is NOT
  called for states that have already been explored. If check_game_over is
  True, returns a boolean value that is True if the game is over, and False
  otherwise.
  """
  if np.all(state != VALUE_BLANK):
    if check_game_over:
      return True
    else:
      return 0.0 # Draw!

  def check_horizontal_wins(state: GameState) -> StateValue:
    for row in state:
      for col in range(len(row) - 3):
        if (row[col:col+4] == row[col]).all() and row[col] in [VALUE_X, VALUE_O]:
          return float(row[col] == VALUE_X) * 2 - 1
    return 0.0 # Ambiguous state.

  horizontal_win = check_horizontal_wins(state)

  if horizontal_win != 0.0:
    if check_game_over:
      return True
    else:
      return horizontal_win
  
  vertical_win = check_horizontal_wins(state.T)

  if vertical_win != 0.0:
    if check_game_over:
      return True
    else:
      return vertical_win

  def check_diagonal_wins(state: GameState) -> StateValue:
    for row in range(len(state) - 3):
      for col in range(len(state[row]) - 3):
        if all([state[row, col] == state[row + offset, col + offset] for offset in range(1, 4)]) and state[row, col] in [VALUE_X, VALUE_O]:
          return float(state[row, col] == VALUE_X) * 2 - 1
    return 0.0 # Ambiguous state.
  
  backslash_diagonal_win = check_diagonal_wins(state)

  if backslash_diagonal_win != 0.0:
    if check_game_over:
      return True
    else:
      return backslash_diagonal_win
  
  slash_diagonal_win = check_diagonal_wins(np.flip(state, axis=1))

  if slash_diagonal_win != 0.0:
    if check_game_over:
      return True
    else:
      return slash_diagonal_win
  
  if check_game_over:
    return False
  else:
    return 0.0

In [0]:
def active_features(state: GameState) -> List[Tuple[int, int, int, LocalFeature]]:
  """
  Takes in a board state and returns the active features.

  For each active feature, we return a tuple consisting of the local feature
  size (width or height), the row index of the top left position within the
  feature, the column index of the top left position within the feature, and
  the feature itself.
  """
  local_features = []

  for local_size in range(1, 6): # local_size is width and height of a feature.
    for row in range(state.shape[0] - local_size + 1):
      for col in range(state.shape[1] - local_size + 1):
        local_feature = state[row : row + local_size, col : col + local_size]

        # Exclude empty features.
        if not np.all(local_feature == VALUE_BLANK):
          # Location-independent feature.
          local_features.append((
              local_size,
              -1,
              -1,
              local_feature,
          ))

          # Location-dependent feature.
          local_features.append((
            local_size,
            row,
            col,
            local_feature,
          ))
  
  return local_features

In [0]:
def encode_feature(feature: LocalFeature) -> EncodedFeature:
  """ Encodes a local feature so that it is hashable for tabular indexing. """
  encoding = 0

  for row in range(feature.shape[0]):
    for col in range(feature.shape[1]):
      encoding += feature[row, col] * 3 ** (row * feature.shape[0] + col)

  return encoding

In [0]:
def drop_piece(state: GameState, column: int, x_move: bool) -> GameState:
  drop_row = state.shape[0] - 1 # In case entire column is empty.

  for row in range(state.shape[0]):
    if state[row][column] != VALUE_BLANK:
      drop_row = row - 1
      break

  new_state = state.copy()
  new_state[drop_row][column] = VALUE_X if x_move else VALUE_O
  
  return new_state

In [0]:
def encode_game_state(state: GameState) -> EncodedFeatureVector:
  encoding = {i: {-1: {-1: []}} for i in range(1, 6)}
  
  for feature in active_features(state):
    encoded_feature = encode_feature(feature[3])
    if not feature[0] in encoding: # Generate local feature size dictionary.
      encoding[feature[0]] = dict()
    if not feature[1] in encoding[feature[0]]: # Generate row dictionary.
      encoding[feature[0]][feature[1]] = dict()
    if not feature[2] in encoding[feature[0]][feature[1]]: # Generate column list.
      encoding[feature[0]][feature[1]][feature[2]] = []
    encoding[feature[0]][feature[1]][feature[2]].append(encoded_feature)
  
  return encoding

In [0]:
weight_vector: WeightVector = {i: {-1: {-1: dict()}} for i in range(1, 6)} # We can already create the row/column keys associated with the location-independent weight vector.

In [0]:
def rollout(agent: "Agent", initial_state: GameState):
  x_player = Agent(True)
  o_player = Agent(False)

  if agent.x_player:
    initial_x_player_encoded_feature_vector = agent._prev_encoded_feature_vector.copy()
    initial_o_player_encoded_feature_vector = encode_game_state(initial_state)
    initial_x_player_state_value = agent._prev_state_value
    initial_o_player_state_value = evaluate(initial_state)
  else:
    initial_x_player_encoded_feature_vector = encode_game_state(initial_state)
    initial_o_player_encoded_feature_vector = agent._prev_encoded_feature_vector.copy()
    initial_x_player_state_value = evaluate(initial_state)
    initial_o_player_state_value = agent._prev_state_value

  rollout_state = initial_state

  while True:
    if agent.x_player:
      rollout_state = x_player.best_move(rollout_state, think_time=0.0, parallel=False)
    else:
      rollout_state = o_player.best_move(rollout_state, think_time=0.0, parallel=False)
    if eval_state(rollout_state, check_game_over=True):
      break
    if agent.x_player:
      rollout_state = o_player.best_move(rollout_state, think_time=0.0, parallel=False)
    else:
      rollout_state = x_player.best_move(rollout_state, think_time=0.0, parallel=False)
    if eval_state(rollout_state, check_game_over=True):
      break

In [0]:
def evaluate(state: GameState) -> float:
  if eval_state(state, check_game_over=True):
    return eval_state(state, check_game_over=False)

  features: List[Tuple[int, int, int, LocalFeature]] = active_features(state)

  accumulator: float = 0
  
  for feature in features:
    # TODO: Implement mirroring / player reversal.
    encoded_feature = encode_feature(feature[3])
    if not feature[0] in weight_vector:
      weight_vector[feature[0]] = dict()
    if not feature[1] in weight_vector[feature[0]]:
      weight_vector[feature[0]][feature[1]] = dict()
    if not feature[2] in weight_vector[feature[0]][feature[1]]:
      weight_vector[feature[0]][feature[1]][feature[2]] = dict()
    if not encoded_feature in weight_vector[feature[0]][feature[1]][feature[2]]:
      weight_vector[feature[0]][feature[1]][feature[2]][encoded_feature] = eval_state(state)

    accumulator += weight_vector[feature[0]][feature[1]][feature[2]][encoded_feature]

  return 1 / (1 + math.exp(-accumulator)) # Sigmoid to squash to [0.0, 1.0].

In [0]:
#@title
class Agent():

  x_player: bool # X player will try to maximize reward, O player will try to minimize it.
  prev_move_rollouts: int
  _prev_encoded_feature_vector: EncodedFeatureVector
  _prev_state_value: float

  def __init__(self, x_player: bool):
    self.x_player = x_player
    self.prev_move_rollouts = 0
    self.reset_game_state()
  
  def reset_game_state(self):
    state = empty_game_state()

    self._prev_encoded_feature_vector = encode_game_state(state)
    self._prev_state_value = evaluate(state)

  def td_update(self,
                new_state: GameState,
                new_state_value: float,
                learning_rate: float=0.1):
    """
    This function could be sped up a lot by cacheing state value, among other
    things.
    """

    feature_count: Dict[int, Dict[int, Dict[int, Counter[EncodedFeature]]]] = {i: {-1: {-1: None}} for i in range(1, 6)}

    signal_power = 0 # Used for normalization to make learning rate invariant under feature variation.
    for local_size in self._prev_encoded_feature_vector:
      for row in self._prev_encoded_feature_vector[local_size]:
        for col in self._prev_encoded_feature_vector[local_size][row]:
          if not row in feature_count[local_size]:
            feature_count[local_size][row] = dict()
          feature_count[local_size][row][col] = Counter(self._prev_encoded_feature_vector[local_size][row][col])
          signal_power += sum([feature_count[local_size][row][col][encoded_feature] ** 2 for encoded_feature in feature_count[local_size][row][col]])
    
    for local_size in feature_count:
      for row in feature_count[local_size]:
        if not row in weight_vector[local_size]:
          weight_vector[local_size][row] = dict()
        for col in feature_count[local_size][row]:
          if not col in weight_vector[local_size][row]:
            weight_vector[local_size][row][col] = dict()
          for encoded_feature in feature_count[local_size][row][col]:
            if not encoded_feature in weight_vector[local_size][row][col]:
              weight_vector[local_size][row][col][encoded_feature] = 0
            weight_vector[local_size][row][col][encoded_feature] += learning_rate * feature_count[local_size][row][col][encoded_feature] / signal_power * (new_state_value - self._prev_state_value)

    # Store values needed for next update.
    self._prev_encoded_feature_vector = encode_game_state(new_state)
    self._prev_state_value = new_state_value
  
  def best_move(self, state: GameState, think_time: float=0.0, epsilon: float=0.15, parallel: bool=True) -> GameState:
    """
    Performs a rollout (if applicable), selects a move, and updates weights.
    Returns the new game state.
    """
    start_time = time()
    if parallel:
      with mp.Pool() as p:
        self.prev_move_rollouts = 0
        for _ in p.imap_unordered(partial(rollout, self), (state for _ in count())):
          self.prev_move_rollouts += 1
          if time() - start_time >= think_time:
            break
    else:
      while time() - start_time < think_time:
        rollout(self, state)

    # Exploitation-Exploration selection.
    if choices([True, False], weights=[epsilon, 1.0-epsilon])[0]:
      # Explore.
      action_column = choice([i for
                              i in
                              range(state.shape[1]) if
                              state[0, i] == VALUE_BLANK])
      new_state = drop_piece(state, action_column, self.x_player)
      self.td_update(new_state, evaluate(new_state))
      return new_state
      
    # Exploit.
    possible_states = [drop_piece(state, column, self.x_player) for
                       column in
                       range(state.shape[1]) if
                       state[0, column] == VALUE_BLANK] # Column is not full of pieces.
    
    if parallel:
      with mp.Pool() as p:
        afterstate_values = p.map(evaluate, possible_states)
    else:
      afterstate_values = [evaluate(possible_state) for possible_state in possible_states]
    
    optimum_value = max(afterstate_values + [0.0]) if self.x_player else min(afterstate_values + [1.0])
    optimum_states = [possible_states[i] for
                      i, afterstate_value in
                      enumerate(afterstate_values) if
                      afterstate_value == optimum_value]
    
    try:
      new_state = choice(optimum_states)
    except IndexError:
      pprint(state)
    self.td_update(new_state, optimum_value)

    return new_state

In [0]:
#@title Reset knowledge

weight_vector: WeightVector = {i: {-1: {-1: dict()}} for i in range(1, 6)}

In [0]:
def optimal_move(state: GameState, x_move: bool) -> GameState:
  scores = requests.get(f"http://kevinalbs.com/connect4/back-end/index.php/getMoves?board_data={''.join(['0'] * 7 + [''.join([str(col) for col in row]) for row in state])}&player={'1' if x_move else '2'}").json()
  return int(max(scores, key=scores.get))

In [76]:
#@title Training by Self-Play

x_player = Agent(True)
o_player = Agent(False)

games =   10000#@param {type: "number"}
think_time = 0#@param {type: "number"}
epsilon = 0.1#@param {type: "number"}
expert_x = False#@param {type: "boolean"}
expert_o = False#@param {type: "boolean"}
show_games = False #@param {type: "boolean"}

parallel = think_time > 0

start_time = time()
def train_game(initial_state: GameState) -> float:
  train_state = initial_state.copy()
  if show_games:
    pprint(train_state)
  while True:
    if show_games:
      print()
    if expert_x:
      train_state = drop_piece(train_state, optimal_move(train_state, True), True)
    else:
      train_state = x_player.best_move(train_state,
                                       think_time=think_time,
                                       epsilon=epsilon,
                                       parallel=parallel)
    if show_games:
      pprint(train_state)
      if not expert_x:
        print(f"Rollouts: {x_player.prev_move_rollouts}")
    if eval_state(train_state, check_game_over=True):
      break
    if show_games:
      print()
    if expert_o:
      train_state = drop_piece(train_state, optimal_move(train_state, False), False)
    else:
      train_state = o_player.best_move(train_state,
                                       think_time=think_time,
                                       epsilon=epsilon,
                                       parallel=parallel)
    if show_games:
      pprint(train_state)
      if not expert_o:
        print(f"Rollouts: {o_player.prev_move_rollouts}")
    if eval_state(train_state, check_game_over=True):
      break
  return eval_state(train_state, check_game_over=False)

if parallel: # `parallel` here refers to parallelization of rollouts and afterstate evaluations.
  for game in range(games):
    result_statement = {
      -1.0: "O wins",
      0.0: "Draw",
      1.0: "X wins",
    }[train_game(empty_game_state())]

    print(f"Game {game+1:,} of {games:,} ({(game+1)/games:.4%}): {result_statement}")
else:
  with mp.Pool() as p:
    for game, result in enumerate(p.imap_unordered(train_game, (empty_game_state() for _ in range(games)),)):
      result_statement = {
        -1.0: "O wins",
        0.0: "Draw",
        1.0: "X wins",
      }[result]

      print(f"Game {game+1:,} of {games:,} ({(game+1)/games:.4%}): {result_statement}")

time_elapsed = time() - start_time

print(f"Played {games:,} games in {time_elapsed:,.2f} seconds.")

Game 1 of 10,000 (0.0100%): O wins
Game 2 of 10,000 (0.0200%): X wins
Game 3 of 10,000 (0.0300%): O wins
Game 4 of 10,000 (0.0400%): O wins
Game 5 of 10,000 (0.0500%): O wins
Game 6 of 10,000 (0.0600%): O wins
Game 7 of 10,000 (0.0700%): O wins
Game 8 of 10,000 (0.0800%): O wins
Game 9 of 10,000 (0.0900%): O wins
Game 10 of 10,000 (0.1000%): O wins
Game 11 of 10,000 (0.1100%): O wins
Game 12 of 10,000 (0.1200%): X wins
Game 13 of 10,000 (0.1300%): X wins
Game 14 of 10,000 (0.1400%): O wins
Game 15 of 10,000 (0.1500%): X wins
Game 16 of 10,000 (0.1600%): X wins
Game 17 of 10,000 (0.1700%): X wins
Game 18 of 10,000 (0.1800%): X wins
Game 19 of 10,000 (0.1900%): X wins
Game 20 of 10,000 (0.2000%): X wins
Game 21 of 10,000 (0.2100%): X wins
Game 22 of 10,000 (0.2200%): X wins
Game 23 of 10,000 (0.2300%): X wins
Game 24 of 10,000 (0.2400%): X wins
Game 25 of 10,000 (0.2500%): X wins
Game 26 of 10,000 (0.2600%): X wins
Game 27 of 10,000 (0.2700%): O wins
Game 28 of 10,000 (0.2800%): O wins
G

Process ForkPoolWorker-2801:
Process ForkPoolWorker-2802:
Process ForkPoolWorker-2805:
Process ForkPoolWorker-2808:
Process ForkPoolWorker-2804:
Process ForkPoolWorker-2806:
Process ForkPoolWorker-2800:
Process ForkPoolWorker-2799:
Process ForkPoolWorker-2797:
Process ForkPoolWorker-2798:
Traceback (most recent call last):
Process ForkPoolWorker-2807:
Traceback (most recent call last):
  File "/home/carter/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/carter/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/carter/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-2803:
  File "/home/carter/anaconda3/lib/python3.7/

Game 3,735 of 10,000 (37.3500%): X wins
Game 3,736 of 10,000 (37.3600%): X wins


  File "<ipython-input-8-d05754fbc837>", line 22, in eval_state
    horizontal_win = check_horizontal_wins(state)
  File "<ipython-input-73-76fab34e5e81>", line 94, in best_move
    afterstate_values = [evaluate(possible_state) for possible_state in possible_states]
  File "<ipython-input-8-d05754fbc837>", line 18, in check_horizontal_wins
    if (row[col:col+4] == row[col]).all() and row[col] in [VALUE_X, VALUE_O]:


KeyboardInterrupt: ignored

In [77]:
mp.cpu_count()

12

In [104]:
#@title Play as O

# Note: Control flow completely does not work for player to play as X yet. Need
# to totally rethink this.

column = "B" #@param ["A", "B", "C", "D", "E", "F", "G"]
think_time =  10#@param {type: "number"}

col_i = ord(column) - ord("A")

try:
  turn = turn
except:
  turn = 1
  playing_as_x = False

print(f"Turn {turn}")

if turn == 1:
  game_state = empty_game_state()

  if playing_as_x:
    game_state = drop_piece(game_state, col_i, x_move=playing_as_x)
  else:
    game_state = x_player.best_move(game_state, think_time=think_time, epsilon=0.01)

  print(f"Win Probability: {evaluate(game_state):.2%}")
  pprint(game_state)

  turn += 1
else:
  if playing_as_x:
    game_state = o_player.best_move(game_state, think_time=think_time, epsilon=0.01)
  else:
    game_state = drop_piece(game_state, col_i, x_move=playing_as_x)

  # Draw. This can only happen after O's turn.
  if np.all(game_state != VALUE_BLANK):
    turn = 1
    print("Draw!")
  elif eval_state(game_state, check_game_over=True):
    pprint(game_state)
    turn = 1
    print("O wins!")
  else:
    if playing_as_x:
      game_state = drop_piece(game_state, col_i, x_move=playing_as_x)
    else:
      game_state = x_player.best_move(game_state, think_time=think_time, epsilon=0.0)

    print(f"Win Probability: {evaluate(game_state):%}")
    pprint(game_state)

    if eval_state(game_state, check_game_over=True):
      turn = 1
      print("X wins!")
    else:
      turn += 1

Turn 9
Win Probability: 50.741974%
 __ _ _ _ _ _ __
1|             |
2|             |
3|      X   O  |
4|    X O X X  |
5|  O O X O X  |
6|X O O X O X  |
 -- - - - - - --
  A B C D E F G 


In [105]:
# See what O's next move should be, according to the agent.
pprint(o_player.best_move(game_state, think_time=2))

 __ _ _ _ _ _ __
1|             |
2|             |
3|      X O O  |
4|    X O X X  |
5|  O O X O X  |
6|X O O X O X  |
 -- - - - - - --
  A B C D E F G 


In [49]:
# Think ahead while O player is thinking of a move...
try:
  o_player.best_move(game_state, think_time=1000000, epsilon=0.33)
except:
  print(f"Rollouts: {o_player.prev_move_rollouts}")

Process ForkPoolWorker-2321:
Process ForkPoolWorker-2325:
Process ForkPoolWorker-2317:
Process ForkPoolWorker-2320:
Process ForkPoolWorker-2322:
Process ForkPoolWorker-2324:
Process ForkPoolWorker-2319:
Process ForkPoolWorker-2318:
Process ForkPoolWorker-2323:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-2327:
Traceback (most recent call last):
  File "/home/carter/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-2328:
Process ForkPoolWorker-2326:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/carter/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/carter/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/h

Rollouts: 4388


In [0]:
save_state = game_state

In [0]:
del turn

In [0]:
game_state = save_state

In [0]:
pprint(game_state)

In [0]:
weight_vector

{1: {-1: {-1: {0: -8.862446973484874e-06,
    1: -4.620145589256038e-06,
    2: -4.41398761962219e-06}},
  0: {0: {0: -4.2610905196102626e-07},
   1: {0: -4.2610905196102626e-07},
   2: {0: -1.4647228244493503e-07, 1: -2.796367695160918e-07},
   3: {0: -4.2610905196102626e-07},
   4: {0: -4.2610905196102626e-07},
   5: {0: -4.2631363698677905e-07,
    1: -2.5713563521791386e-12,
    2: 2.0715638210503922e-10},
   6: {0: 1.813819578875515e-05,
    1: 1.1749802516363788e-10,
    2: -1.8564422338741315e-05}},
  1: {0: {0: -4.260889617488989e-07, 2: -2.0090212127232833e-11},
   1: {0: 1.8417834417591785e-05,
    1: 5.632092402053226e-12,
    2: -1.8843949101645192e-05},
   2: {0: -1.4662473269126357e-07,
    1: 5.0689960049356715e-11,
    2: -2.7953500922981177e-07},
   3: {0: -4.2610905196102626e-07},
   4: {0: -4.2610905196102626e-07},
   5: {0: -1.4652048883176154e-07, 1: -2.7958856312926565e-07},
   6: {0: -1.464058091818356e-07,
    1: -2.795469478717e-07,
    2: -1.5629490749165737e-

# Algorithm

- [X] Start with a game state $s_t$.
- [X] For the afterstate of each possible action:
  - [X] Enumerate the local shape features.
  - [X] Look up each local shape feature in the weights table.
    - [X] Implement location dependence.
  - [X] Sum all the weights associated with each feature up.
  - [X] Apply the logistic sigmoid function to this linear sum. The result of this calculation is the associated action's afterstate value.
    - $V(s) = \sigma(\phi(s) \cdot \theta^{LI} + \phi(s) \cdot \theta^{LD})$
- [X] Select the next action which results in the greatest afterstate value $V(s_{t+1})$.
  - [X] Other player tries to minimize $V(s_{t+1})$ instead of maximize it.
  - [X] $\epsilon$-greedy selection
- [X] Save what you need such that you can do the following update after your opponent makes a move and creates the new state $s_{t+2}$:
  - [X] $\Delta\theta^{LD} = \Delta\theta^{LI} = \alpha \frac{\phi(s_t)}{||\phi(s_t)||^2}(V(s_{t+2})-V(s_t))$
  - [X] Save $\phi(s_t)$ and $V(s_t)$, as well as a reference to $\theta_t$ so that it can be updated by $\Delta\theta$.
